In [1]:
import pandas as pd
import numpy as np

Substitute for own data path

In [2]:
df = pd.read_csv(r'C:\Users\eric7\OneDrive\Desktop\aps360Project\convolutional-neural-nuts\data\set.csv')
print("successfully retrieved csv file")

successfully retrieved csv file


IN CSV files, formate as [path:genus]
Created a dictionary that has keys as genus, and items as image paths

In [3]:
grouped = df.groupby('Genus')['Path'].apply(list).to_dict()

mapping = {key: np.array(value) for key, value in grouped.items()}

Analysis of data
- help decide cnn approach
- all data in a numpy dictionary called mapping

In [4]:
print(f"number of classes: {len(mapping)}")

number of classes: 70


In [6]:
for dict_keys in mapping.keys():
    print(f"Class name: {dict_keys}, number of images: {len(dict_keys)}")

Class name: Amanita, number of images: 7
Class name: Amauroderma, number of images: 11
Class name: Antrodia, number of images: 8
Class name: Aurantiopileus, number of images: 14
Class name: Auricularia, number of images: 11
Class name: Boletinellus, number of images: 12
Class name: Brunneocorticium, number of images: 16
Class name: Calocera, number of images: 8
Class name: Calvatia, number of images: 8
Class name: Camillea, number of images: 8
Class name: Chlorophyllum, number of images: 13
Class name: Clavaria, number of images: 8
Class name: Cookeina, number of images: 8
Class name: Coprinellus, number of images: 11
Class name: Cordierites, number of images: 11
Class name: Coriolopsis, number of images: 11
Class name: Cotylidia, number of images: 9
Class name: Cyathus, number of images: 7
Class name: Cymatoderma, number of images: 11
Class name: Cyptotrama, number of images: 10
Class name: Dacryopinax, number of images: 11
Class name: Dicephalospora, number of images: 14
Class name: 